<a href="https://colab.research.google.com/github/Hideyuki-Machida/Python-tinychain/blob/main/tinychain_UTXO_POW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tinychain-UTXO-POW

### ■ UTXO （Unspent Transaction Output）

未使用のトランザクションアウトプットの合計で残高を管理するモデル。</br>
ブロックチェーンなどに使用されている。</br>
対比はイーサリアムなどで使用される "アカウントモデル"。</br>
</br>

### ■ POW （Proof of Work）

トランザクション承認の合意形成モデル。</br>
トランザクション承認の権利を計算競争で争い、勝者はの承認報酬を得る。</br>
</br>
</br>
↓ プログラムの原型・テキストは、こちらから参照・引用しています。</br>
[openreachtech/tinychain](https://github.com/openreachtech/tinychain/tree/main/pow)

#### Tinycoin の仕様解説

---

実際の Bitcoin と異なる簡略化した部分を中心に解説します。

* １つのトランザクションで１つのコインを送る
  * 通常のトランザクションは、１つのトランザクションで複数のコインを送れます。また、複数人に送ることも可能です。しかし、Tinycoin では簡略化して自分の持ってる１枚のコインを誰かに送るとします
* POW 風なコンセンサス
  * Bitcoin の POW をそのまま実装すると、コンピューターリソースを消費してしまうので、POW を模したような実装をします。8 秒に１回だけランダムにブロックが生成されます。
* １つの Node が集権的にブロックを生成する
  * ブロックチェーンは分散性が肝です。今回は簡略化してとある Node が集権的にブロックを生成するとします。

参考 : </br>
*  [wikipedia - Unspent transaction output](https://en.wikipedia.org/wiki/Unspent_transaction_output)
*  [「UTXO」ブロックチェーンの取引データをひとつなぎにする仕組み](https://gaiax-blockchain.com/utxo)
*  [wikipedia - Proof of work](https://en.wikipedia.org/wiki/Proof_of_work)
*  [Practical Cryptographic for Developers - Python Crypto Libraries - ECDSA](https://cryptobook.nakov.com/crypto-libraries-for-developers/python-crypto-libraries)




## Setup

#### eth-keysをインストール

In [1]:
!pip install eth-keys
!pip install eth-hash[pycryptodome]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Module

### Step1: Wallet を作る

今回作る Wallet は秘密鍵を持ち、トランザクションに署名するだけの簡単なものです。
`EC.genKeyPair()`で秘密鍵を生成して、`key.sign(tx.hash)`でトランザクションに署名します。
Bitcoin のアドレスは公開鍵からとある関数を通すことで生成されますが、今回は、公開鍵をそのままアドレスとして代用します。


In [2]:
import eth_keys

# ---------------------------------------------------------------------------------
# コード

class Wallet:
    def __init__(self, key):
        self.privKey = eth_keys.keys.PrivateKey(key * 32)
        self.pubKey = self.privKey.public_key

    # トランザクションに署名
    def signTx(self, tx):
        signature = self.privKey.sign_msg(tx.hash)
        tx.inSig = signature
        return tx

# ---------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------
# 動作確認

aliceWallet = Wallet(b"\x01")
bobWallet = Wallet(b"\x02")

print("Alice privKey : ", aliceWallet.privKey)
print("Alice pubKey : ", aliceWallet.pubKey)

print("Bob privKey : ", bobWallet.privKey)
print("Bob pubKey : ", bobWallet.pubKey)

# ---------------------------------------------------------------------------------

Alice privKey :  0x0101010101010101010101010101010101010101010101010101010101010101
Alice pubKey :  0x1b84c5567b126440995d3ed5aaba0565d71e1834604819ff9c17f5e9d5dd078f70beaf8f588b541507fed6a642c5ab42dfdf8120a7f639de5122d47a69a8e8d1
Bob privKey :  0x0202020202020202020202020202020202020202020202020202020202020202
Bob pubKey :  0x4d4b6cd1361032ca9bd2aeb9d900aa4d45d9ead80ac9423374c451a7254d07662a3eada2d0fe208b6d257ceb0f064284662e857f57b66b54c198bd310ded36d0


### Step2: Transaction を作る

トランザクションは`未消費のトランザクションのハッシュ値（inHash）`、`署名（sig）`、`受取り手アドレス（outAddr）`、`トランザクションのハッシュ値（hash）`の４要素で構成されます。


In [3]:
import hashlib
import json

# ---------------------------------------------------------------------------------
# コード

class Transaction:
    def __init__(self, inHash, outAddr, sig = ""):
        self.inHash = inHash
        self.inSig = sig
        self.outAddr = outAddr
        self.hash = Transaction.hash(inHash, outAddr)

    def toString(self):
        obj = { 
            "inHash": self.inHash.decode(), 
            "inSig": self.inSig.to_hex(), 
            "outAddr": self.outAddr.to_hex(), 
            "hash": self.hash.decode()
        }
        return json.dumps(obj)

    @staticmethod
    def hash(inHash, outAddr):
        hash = hashlib.sha256( "{0},{1}".format(inHash, outAddr).encode() ).hexdigest().encode()
        return hash
# ---------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------
# 動作確認

# マイニングでの報酬トランザクション （ コインベーストランザクション ）
tx001 = Transaction(b"", aliceWallet.pubKey)
print("tx001_hash : ", tx001.hash)

# 送信元のaliceWalletでトランザクションに署名
tx001 = aliceWallet.signTx(tx001)
print("tx001_signature : ", tx001.inSig)

# aliceWalletで署名されたトランザクションハッシュをaliceWalletで検証
print("verify_msg001 : ", tx001.inSig.verify_msg(tx001.hash, aliceWallet.pubKey))

# aliceWallet から bobWalletに transfer するトランザクション
tx002 = Transaction(tx001.hash, bobWallet.pubKey)
print("tx002_hash :", tx002.hash)

# 送信元のaliceWalletでトランザクションに署名
tx002 = aliceWallet.signTx(tx002)
print("tx002_signature : ", tx002.inSig)

# aliceWalletで署名されたトランザクションハッシュをaliceWalletで検証
print("verify_msg002 : ", tx002.inSig.verify_msg(tx002.hash, aliceWallet.pubKey))

# aliceWalletで署名されたハッシュをbobWalletで検証（署名検証失敗）
print("verify : ", bobWallet.pubKey.verify_msg(tx001.hash, tx001.inSig))
print("verify : ", bobWallet.pubKey.verify_msg(tx001.hash, tx002.inSig))


# ---------------------------------------------------------------------------------

tx001_hash :  b'1b13b3f09a4bfb51c5c31bdc3ecce9ee7c8c575f6b5f504f9bd8d79b2060e271'
tx001_signature :  0x117afa1607a32d6002dd043b7867a34176fd20355ebc0098083f79aa5bd0ca0620d8705a4a83274e880e93e3a7d28b219e3a0623b8dbbe7d2d47a22251120e3201
verify_msg001 :  True
tx002_hash : b'7b45172d124b6abec761bb98d36d458b9a36cb0a06506e4de52a45c08d134204'
tx002_signature :  0x90369318f27e511f540a48319557c6ead94aebfeddbdd2eb299642b2fc84f6521bb7fc2f9372f5824c816d63d53258a189e90ab2f317aac489e55ba05872211301
verify_msg002 :  True
verify :  False
verify :  False


### Step3: Transaction Pool へ Transaction を追加する

ブロックチェーンの Node に送られたトランザクションはトランザクションプールに格納されます。
トランザクションプールは「まだブロックに取り込まれていないトランザクションの集合」です。
後々、ブロックに追加されますが、まずは、トランザクションプールにプールされます。


In [4]:
# ---------------------------------------------------------------------------------
# コード

class TxPool:
    def __init__(self):
        self.txs = [] # 未承認トランザクション
        self.unspentTxs = [] # 未消費トランザクション

    def addTx(self, tx):
        try:
            TxPool.validateTx(self.unspentTxs, tx);
            self.txs.append(tx);
        except Exception as e:
            print("🍎　error: ", e)

    def balanceOf(self, address):
        return len(list(filter(lambda tx: tx.outAddr == address, self.unspentTxs)))

    def updateUnspentTxs(self, spentTxs):
        for tx in spentTxs:
            spentHash = tx.inHash
            ttx = list( filter(lambda unspentTx: unspentTx.hash == spentHash, self.unspentTxs) )
            if len(ttx) == 0: 
                return
            else:
                self.unspentTxs.remove( ttx[0] )
        self.unspentTxs.extend(spentTxs)


    @staticmethod
    def validateTx(unspentTxs, tx):
        # hash値が正しく計算されているかチェック
        if tx.hash != Transaction.hash(tx.inHash, tx.outAddr):
            raise Exception("invalid tx hash. expected: {0}".format(Transaction.hash(tx.inHash, tx.outAddr)))
            #throw new Error(`invalid tx hash. expected: ${Transaction.hash(tx.inHash, tx.outAddr)}`);

        # 消費済みトランザクションではないか（未消費のトランザクションであること確認）チェック
        inTx = list( filter(lambda unspentTx: unspentTx.hash == tx.inHash, unspentTxs) )

        if len(inTx) == 0:
            raise Exception("tx in not found")

        if TxPool.validateSig(tx, inTx[0].outAddr) != True:
            raise Exception("invalid tx hash. expected:")

    @staticmethod
    def validateSig(tx, address):
        return tx.inSig.verify_msg(tx.hash, address)

# ---------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------
# 動作確認

pool = TxPool()

# マイニングでの報酬トランザクション （ コインベーストランザクション ）
tx001 = aliceWallet.signTx(Transaction(b"", aliceWallet.pubKey))

# tx001で未消費トランザクションを更新
pool.unspentTxs.append(tx001)

# aliceWalletのBalanceが1になる。
print("aliceWalletBalance : ", pool.balanceOf(aliceWallet.pubKey))
print("bobWalletWalletBalance : ", pool.balanceOf(bobWallet.pubKey))

# aliceWallet から bobWalletに transfer するトランザクションを作成
tx002 = aliceWallet.signTx(Transaction(tx001.hash, bobWallet.pubKey))

# tx002をpoolに追加（マイニングによる承認待ち）
pool.addTx( tx002 )

# poolの承認待ちトランザクションで未消費トランザクションを更新
spentTxs = pool.txs
pool.tx = []
pool.updateUnspentTxs( spentTxs )

# aliceWalletのBalanceが0になり、bobWalletのBalanceが1になる。
print("aliceWalletBalance : ", pool.balanceOf(aliceWallet.pubKey))
print("bobWalletWalletBalance : ", pool.balanceOf(bobWallet.pubKey))

# ---------------------------------------------------------------------------------

aliceWalletBalance :  1
bobWalletWalletBalance :  0
aliceWalletBalance :  0
bobWalletWalletBalance :  1


### Step4: Block を Transaction Pool 内のトランザクションから作る

ブロックチェーンのブロックは、トランザクションの容器のようなものです。トランザクションプール内のトランザクションのうち、手数料を割高に設定されたものから優先的に抽出されます。
今回は手数料の概念を省略しているので、トランザクションプール内の全てを取り込む実装にしています。

通常、ブロックはブロックヘッダーと中身に分けられるのですが、今回は簡略化のため分けていません。
ブロックは`ブロック高さ(height)`、`直前ブロックのハッシュ値(preHash)`、`タイスタンプ(timestamp)`、`トランザクションデータ(data)`、`ノンス(nonce)`、`ブロックのハッシュ値(hash)`の６要素で構成されます。


In [10]:
import datetime

# ---------------------------------------------------------------------------------
# コード

class Block:
    def __init__(self, height, preHash, timestamp, data, nonce):
        self.height = height;
        self.preHash = preHash;
        self.timestamp = timestamp;
        self.data = data;
        self.nonce = nonce;
        self.hash = Block.hash(height, preHash, timestamp, data, nonce);

    @staticmethod
    def hash(height, preHash, timestamp, data, nonce):
        hash = hashlib.sha256( "{0},{1},{2},{3},{4}".format(height, preHash, timestamp, data, nonce).encode() ).hexdigest()
        return hash

# ---------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------
# 動作確認

now = int(datetime.datetime.now().timestamp())
# 最初のBlockを作成
pre = Block(0, "0", now, "{}", 0)
print("Block.hash : ", pre.hash)

# ---------------------------------------------------------------------------------

Block.hash :  e67922d0534611a0a99d3a8cb6e2e8318b7d0236faf3f50e8a80de94e04eae75


### Step5: マイニング

マイニングを簡略化した形で実装しました。
Nonce をインクリメンとして、hex 表記のブロックのハッシュ値の先頭に、ゼロが difficulty 個以上並ぶか確かめる作業を、１秒間に 32 回行います。difficulty が 2 の場合、ブロックハッシュの先頭にゼロが２つ並ぶ確率は、1/256 なので、８秒に一回の確率で正しいブロックが生成されます。


In [6]:
import time
from functools import reduce

# ---------------------------------------------------------------------------------
# コード

def mining(nonce, difficulty, txs, pre, conbaseTx):
    data = conbaseTx.toString()
    for tx in txs:
        data += tx.toString()

    now = int(datetime.datetime.now().timestamp())
    block = Block(pre.height + 1, pre.hash, now, data, nonce);
    print("block.hash : ", block.hash)

    # hash値のhexの先頭に'0'が'difficulty'個以上つけば正規のブロックになる
    if block.hash.startswith("0"*difficulty):
        return True, block
    else:
        return False, block

# ---------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------
# 動作確認

nonce = 0
difficulty = 2

# コインベーストランザクションを作成
conbaseTx = aliceWallet.signTx( Transaction(b"", aliceWallet.pubKey) )
txs = []
pre = Block(0, "0", now, "{}", 0)
isComplete = False
while isComplete == False:

    # マイニング
    isComplete, block = mining(nonce, difficulty, txs, pre, conbaseTx)
    if isComplete == True:
        print("🍏 complete")
        print("block : ", block)
    else:
        nonce += 1 # nonceをインクリメントすることで、hash値に変化をつける
        print("nonce : ", nonce)

        # difficulty=2の場合、先頭にゼロが２つ揃う確率は、1/256 （MAX 256回でマイニング成功するということではない）
        time.sleep(1 / 32) # 1秒に32回試行するなら、256/32=8秒に一回だけブロックを生成する

# ---------------------------------------------------------------------------------

block.hash :  4c4b84131fffc59ce3ede633547c6f9a7119366492d8ae86e1595c7092093294
nonce :  1
block.hash :  9a169bee39fb21a915f0979c7581da020785874b00e606f6da972c566725a55a
nonce :  2
block.hash :  b260849ab8a982fd7fb081cddb64fc268440d1b4f7d6bd32447ade528f06d2f4
nonce :  3
block.hash :  77434d14063e94aa214b9c06b2492b55e293c0969c2e68c5168c64877a1f8663
nonce :  4
block.hash :  ce760ca01174081dbe42f380571c334da9efb6bebfe6fb77332df59cdbed9dc8
nonce :  5
block.hash :  e68364b37f9241573c1479643a4b5f7d6c20d8eddcc1cfe1bb3e57e2c7b88ca2
nonce :  6
block.hash :  cb0dc22075c516d674ed61aa4e23ab8622fbc2bfb3cb4b34999a044270c2642f
nonce :  7
block.hash :  65fd35c2f8075da07d217cce3b55856849a329d215e0055322ae870295fb85a5
nonce :  8
block.hash :  d4cef2358b7fe013f80c8ea8604dce42be8ad30283c22e6cc906345114e34749
nonce :  9
block.hash :  29ec0893c5bf9133b856bd16fdddd4b07dc7b31011a55a11fa043544a31425a7
nonce :  10
block.hash :  8e4adace5944b6a386630de42acaa3aeb8988c3314d46805cf5f192e8a4454c0
nonce :  11
block.ha

### Step6: Block をチェーンへ追加

マイニングを行い、正しいブロックを発見したらそのブロックをチェーンに追加します。
具体的には`blocks`という配列に追加します。
通常のブロックチェーンでは正しいブロックを見つけたら、P2P で繋がっている別の Node に伝播します。今回はブロックチェーン Node は１つか存在さず、集権的にブロックを積み上げるものとしました。


In [7]:
import datetime

# ---------------------------------------------------------------------------------
# コード

class Tinycoin:
    def __init__(self, wallet, difficulty = 2):
        now = int(datetime.datetime.now().timestamp())
        self.blocks = [ Block(0, "0", now, "{}", 0) ]
        self.pool = TxPool();
        self.wallet = wallet # コインベースTxを受け取るウォレット
        self.difficulty = difficulty
        self.stopFlg = False

    def latestBlock(self):
        return self.blocks[ len(self.blocks) - 1 ]

    def addBlock(self, newBlock):
        self._validBlock(newBlock);
        self.blocks.append(newBlock);

    def _validBlock(self, block):
        preBlock = self.latestBlock()
        expHash = Block.hash(block.height, block.preHash, block.timestamp, block.data, block.nonce)
        if preBlock.height + 1 != block.height:
            # ブロック高さが直前のブロックの次であるかチェック
            raise Exception("invalid heigh. expected: {0}".format(preBlock.height + 1))
        elif preBlock.hash != block.preHash:
            # 前ブロックハッシュ値が直前のブロックのハッシュ値と一致するかチェック
            raise Exception("invalid preHash. expected: {0}".format(preBlock.hash))
        elif expHash != block.hash:
            # ハッシュ値が正しいく計算されているかチェック
            raise Exception("invalid hash. expected: {0}".format(expHash))
        elif block.hash.startswith("0"*self.difficulty) != True:
            # difficultyの要件を満たすかチェック
            raise Exception("invalid hash. expected to start from {0}".format("0"*self.difficulty))

    async def genNexBlock(self):
        nonce = 0
        pre = self.latestBlock()
        conbaseTx = self._genCoinbaseTx()
        txs = self.pool.txs
        isComplete = False
        while isComplete == False:
            # マイニング
            isComplete, block = mining(nonce, self.difficulty, txs, pre, conbaseTx)
            if isComplete == True:
                print("🍏 complete")
                # 全てのUtxoがブロックに取り込まれたとしtxpoolを空にする
                spentTxs = self.pool.txs
                self.pool.txs = []
                self.pool.updateUnspentTxs(spentTxs)
                self.pool.unspentTxs.append(conbaseTx)
                return block
            else:
                nonce += 1 # nonceをインクリメントすることで、hash値に変化をつける
                print("nonce : ", nonce)

                # difficulty=2の場合、先頭にゼロが２つ揃う確率は、1/256 （MAX 256回でマイニング成功するということではない）
                time.sleep(1 / 32) # 1秒に32回試行するなら、256/32=8秒に一回だけブロックを生成する


    async def startMining(self):
        block = await self.genNexBlock()
        self.addBlock(block)
        print("🍏 new block mined! block number is {0}".format(block.height));

    def _genCoinbaseTx(self):
        # minerへの報酬として支払われるコインベーストランザクション
        # inputがなくて、outputがminerのウォレット
        return self.wallet.signTx( Transaction(b"", self.wallet.pubKey) )

# ---------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------
# 動作確認

tinycoin = Tinycoin(wallet=aliceWallet)

try:
    # マイニング開始
    await tinycoin.startMining();
    print("完了")

except Exception as e:
    print("🍎　error: ", e)

print("aliceWalletBalance : ", tinycoin.pool.balanceOf(aliceWallet.pubKey))
print("bobWalletWalletBalance : ", tinycoin.pool.balanceOf(bobWallet.pubKey))

# ---------------------------------------------------------------------------------

block.hash :  7381f5d92744715a5161045eedc9fde0f1e69a5d7f18477743d84494c17307e6
nonce :  1
block.hash :  655947fcbd0d67b3ebb4ba248276437184d2c5ba9cd6e47605690f17686e0ac2
nonce :  2
block.hash :  c9fdcead3c35a2920c9bcc16f73af20cfb26227ec9ea6db48bf82881cc548571
nonce :  3
block.hash :  305640061d143fd2cd66fc7a96b1c10091fd873dae1a383a34ccccc75bdf50ef
nonce :  4
block.hash :  80cdbda98029cf7759f0ac94e653a8f91b581c6ea581fa5f01a249e2ee2f8b8a
nonce :  5
block.hash :  7990f8042e7295c17e4f425f026a9f9f684bd2c62507361c3a4bee5c66d499e9
nonce :  6
block.hash :  e72c5de7c4a6f8fa6b010e888bfe38c5a9ba3cb612afa700145cfdfc8b2e987e
nonce :  7
block.hash :  15d61a526ce1dcba10efd0ba4d0d9997e583af4379a5217cda50dc7518f2c716
nonce :  8
block.hash :  e0371a4520d945ba045aad254b1e1b2039635c1c79616834eea1e63bac202e1a
nonce :  9
block.hash :  dc2312738f1472e21b8e44446dd23d4067cce8a6473e64d0ba4a73e0f321368f
nonce :  10
block.hash :  759e9dc08bb566b1fd3d5b4b3be281d579310a9ce380f20bfa111a7ad0439a0c
nonce :  11
block.ha

In [8]:
# ---------------------------------------------------------------------------------
# コード

def transfer(wallet, outAddr):
    # wallet の未消費トランザクションを確認
    result = list(filter(lambda tx: tx.outAddr == wallet.pubKey, tinycoin.pool.unspentTxs))
    if len(result) == 0:
        print("no available unspent transaction")

    # transferトランザクションを作成
    tx = wallet.signTx( Transaction(result[0].hash, outAddr) )

    # transferトランザクションをpoolに追加（マイニングによる承認待ち）
    tinycoin.pool.addTx( Transaction(tx.inHash, tx.outAddr, tx.inSig) )

# ---------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------
# 動作確認

try:
    # aliceWallet から bobWalletに transfer
    transfer(wallet=aliceWallet, outAddr=bobWallet.pubKey)

    # マイニング開始
    await tinycoin.startMining()
    print("完了")
except Exception as e:
    print("🍎　error: ", e)


# aliceWalletのBalance が transferで-1、マイニング報酬で+1 bobWalletのBalanceがtransferで+1
print("aliceWalletBalance : ", tinycoin.pool.balanceOf(aliceWallet.pubKey))
print("bobWalletWalletBalance : ", tinycoin.pool.balanceOf(bobWallet.pubKey))

# ---------------------------------------------------------------------------------

block.hash :  3ce8cbf64b3f848e5777502b1129aa42a12391f8a660ed765aa648c8196a65b7
nonce :  1
block.hash :  96806f68a2b31458e73d95682afe94c4be76f049b5b6d1b008ea545227f6ff3a
nonce :  2
block.hash :  36be53508da1f21bad7e145a8c0aa76020e460a3771e8f6ef10fbf13368d89f5
nonce :  3
block.hash :  5abb19c9ac4b4679a1e3af5ce44da99df5c654bac8820185a152cd204bb0f8f1
nonce :  4
block.hash :  ef314404e4c793f82edc4b6602db900cd7b73e9cc205cf0279a0326806225b12
nonce :  5
block.hash :  1f477a62aab87f85cbeaab2df23809d9840871c189cb02b242a59faf51fdea58
nonce :  6
block.hash :  09f077d48ae26928206c4e29f0feba6fbf3d5d4e83d56b972453f479fe2a5186
nonce :  7
block.hash :  494c4403cb324edad9f7a767e84d89af911f05bfe885d07167f6bf703049ef5c
nonce :  8
block.hash :  9745643df963e8893ef5b9454b92ccf0297e5d0bd80886061d9e4c589ea4e865
nonce :  9
block.hash :  9b957cf350e08f1ed0a57e68c45b818c60da5a50c3ab8a7759f7e3efb64efa53
nonce :  10
block.hash :  690095e7efab71a6cfb95295eb7772d2c525548077f6e02b0da8ade8b0b6a37b
nonce :  11
block.ha